<a href="https://colab.research.google.com/github/tytttyyyttyy/net_learning/blob/main/auto_cer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[record_sheet](https://docs.google.com/spreadsheets/d/1poMhLbeQSrMNnJgTW5zS1bi_hqf6PnHnEi_2aTJ8kRc/edit#gid=0)

[template_sheet
](https://docs.google.com/spreadsheets/d/1mQ2WUiGKHUJsQBuBJHzOhSst_lsIwYfmZE6y81bz2K4/edit?usp=drive_link)

#Prepare

In [ ]:
!pip install gspread

import pandas as pd
import gspread
import requests
from oauth2client.service_account import ServiceAccountCredentials

from google.colab import auth
auth.authenticate_user()

import googleapiclient.discovery
import google.auth
from google.auth.transport.requests import Request
from oauth2client.client import GoogleCredentials
from gspread import authorize

creds, _ = google.auth.default()
gc = authorize(creds)

In [ ]:
file_path = '/content/drive/My Drive/auto certificate/HR_Dataset_2024.csv'

df = pd.read_csv(file_path) # 讀取 CSV 檔案

## 取得 Google Spreadsheets 工作表 : record_sheet
record_sheet_id = '1poMhLbeQSrMNnJgTW5zS1bi_hqf6PnHnEi_2aTJ8kRc'
record_sheet_name = 'record'
record_sheet = gc.open_by_key(record_sheet_id).worksheet(record_sheet_name)


# AUTO Certificate

###function

In [ ]:
#-------------------------------------------分隔線------------------------------------------------------#
## def挑選employee_num

def choose_empnum():
    df['Emp #'] = df['Emp #'].astype(str)  ##將 Emp 欄位轉換為文字格式
    while True:  ## 使用 while 迴圈讓使用者可以重複輸入
        emp_num = input("請輸入要查詢的Emp #：")  ## 讓使用者輸入特定 Emp
        df_filtered = df[df['Emp #'] == emp_num]  ## 篩選出符合條件的資料

        ## 判斷是否有資料
        if len(df_filtered) > 0:
            ## 取得員工資料
            employee_data = df_filtered[['Emp #', 'Employee Name', 'Location', 'National ID','Department','DOB', 'Position', 'Date of Termination']].to_dict(orient='records')[0]

            ## 將 NaN 轉換為字串 'nan'
            for key, value in employee_data.items():
                if pd.isna(value):
                    employee_data[key] = 'nan'

            today = pd.Timestamp('today')  ## 取得目前日期
            today_str = today.strftime('%Y-%m-%d')
            employee_data['Current Date'] = today_str  ## 將目前日期加到字典中

            ## 輸出員工資料
            data = pd.DataFrame([employee_data]).values.tolist()  ## 將DataFrame轉換為二維列表
            data = data[0]  ## 將列表轉換為一維
            record_sheet.append_row(data)  ## 將data導入record sheet

            return employee_data, data, emp_num

        else:
            print(f"查無Emp # {emp_num} 的資料")

            ## 詢問使用者是否要重新輸入
            choice = input("是否要重新輸入？ (y/n)：")
            if choice.lower() != 'y':
                break  ## 若使用者輸入不是 'y'，則結束迴圈

#-------------------------------------------分隔線------------------------------------------------------#
## def複製工作表
  ##複製template工作表到copied工作表，source_sheet_id 為要複製的工作表的 ID，new_sheet_name 為複製工作表的名稱。

def duplicate_sheet(spreadsheet_id, source_sheet_id, new_sheet_name):
  #delete_sheet(spreadsheet_id, new_sheet_name)
  sheets_service = googleapiclient.discovery.build('sheets', 'v4')
  request = {
      'requests': [
          {
              'duplicateSheet': {
                  'sourceSheetId': source_sheet_id,
                  'newSheetName': new_sheet_name,
              }
          }
      ]
  }
  sheets_service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=request).execute()

  template_sheet = gc.open_by_key(spreadsheet_id).worksheet(new_sheet_name)
  return template_sheet

#-------------------------------------------分隔線------------------------------------------------------#
## 替換模板資料到copied工作表
def data_to_copied(employee_data, data, template_sheet):

  employee_data_columns = pd.DataFrame([employee_data]).columns.tolist()
  a = pd.DataFrame([employee_data]).shape[1]  ## a等於employee_data_columns數量

  ## 從0到a都會執行一次
  for i in range(a):
      search_term = "{{ [" + employee_data_columns[i] + "] }}"
      replace_term = data[i]

      for row_index, row in enumerate(template_sheet.get_all_values()):
          for column_index, cell in enumerate(row):
              if search_term in cell:
                  ## 找到搜尋詞彙所在的列和欄位
                  search_term_column = column_index
                  search_term_row = row_index + 1

                  ## 更新指定的欄位
                  template_sheet.update_acell(f'{chr(ord("A") + search_term_column)}{search_term_row}', replace_term)

  copied_id = template_sheet.id
  return copied_id

#-------------------------------------------分隔線------------------------------------------------------#
## def生成pdf

def generate_pdf(template_sheet_id, copied_id, emp_num):
    ## google sheet to PDF #url範例:https://docs.google.com/spreadsheets/d/<spreadsheet_id>/export?format=pdf&gid=<worksheet_id>
    pdf_url = f"https://docs.google.com/spreadsheets/d/{template_sheet_id}/export?format=pdf&gid={copied_id}"
    response = requests.get(pdf_url)
      ## Construct the desired file path
    pdf_file_name = f"termination_certificate_{emp_num}.pdf"
    pdf_file_path = "/content/drive/My Drive/auto certificate/outputs/" + pdf_file_name

    ## Save the PDF to the specified location
    with open(pdf_file_path, 'wb') as f:
        f.write(response.content)
    print(f"PDF saved to {pdf_file_path}")

#-------------------------------------------分隔線------------------------------------------------------#
## def刪除copied工作表
def delete_sheet(template_sheet_id, template_name):

  try:## Open the template
    spreadsheet = gc.open_by_key(template_sheet_id)
    worksheet = spreadsheet.worksheet(template_name)
  except (KeyError, AttributeError) as e:
    print(f"Error accessing worksheet '{template_name}': {e}")
    exit(1)

  ## Delete the worksheet
  try:
    spreadsheet.del_worksheet(worksheet)
    print(f"Worksheet '{template_name}' deleted from spreadsheet with ID '{template_sheet_id}'")
  except Exception as e:
    print(f"Failed to delete worksheet '{template_name}': {e}")
    exit(1)

#-------------------------------------------分隔線------------------------------------------------------#




### execute

In [ ]:
template_sheet_id = '1mQ2WUiGKHUJsQBuBJHzOhSst_lsIwYfmZE6y81bz2K4'
template_name = 'copied'
employee_data, data, emp_num = choose_empnum() ## def-挑選employee_num
template_sheet = duplicate_sheet(template_sheet_id, '0', template_name ) ## def-複製工作表
copied_id = data_to_copied(employee_data, data, template_sheet) ## def-替換模板資料到copied工作表
generate_pdf(template_sheet_id, copied_id, emp_num) ## def-生成pdf
delete_sheet(template_sheet_id, template_name) ## def-刪除copied工作表

請輸入要查詢的Emp #：E0043
PDF saved to /content/drive/My Drive/auto certificate/outputs/termination_certificate_E0043.pdf
Worksheet 'copied' deleted from spreadsheet with ID '1mQ2WUiGKHUJsQBuBJHzOhSst_lsIwYfmZE6y81bz2K4'
